# 01/20

%matplotlib inline

In [1]:
description = 'train_mobilenet_from_pretrain_v3_add_valence_arousal'

In [2]:

import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import cv2
from PIL import Image
from random import shuffle
import math

import pandas as pd

import pickle

from sklearn.svm import SVC,LinearSVC
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier,ExtraTreesClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import preprocessing

#from scipy.misc import imread, imresize



In [3]:
import tqdm
import json

In [4]:
train_df = pd.read_csv('../../data/Manually_Annotated_file_lists/training_face_mesh_crop.csv')
train_df.subDirectory_filePath = '../../data/Manually_Annotated_Images_FaceMesh_Cropped/' + train_df.subDirectory_filePath

#train_df_2 = pd.read_csv('../../data/Automatically_annotated_file_list/automatically_annotated_face_mesh_crop.csv')
#train_df_2.subDirectory_filePath = '../../data/Automatically_Annotated_Images_FaceMesh_Cropped/' + train_df_2.subDirectory_filePath
#train_df = train_df.append(train_df_2)
#del train_df_2

train_df = train_df[train_df['have_facemesh']]

In [5]:
from eason_utils import DataFrameBatchIterator
from eason_utils import lprint, now_time_string, log_file_name, change_log_file_name

change_log_file_name(f'''{log_file_name.replace('.log', '')}_{description}.log''')

log_file_name='01_22_23:41:48.log'


In [6]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import tensorflow as tf
config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(allow_growth = True))
sess = tf.compat.v1.Session(config = config)

In [7]:
mobilenet_pretrained = tf.keras.models.load_model("../models/affectnet_emotions/mobilenet_7.h5")

In [8]:
mobilenet_pretrained.trainable = False
mobilenet_output = mobilenet_pretrained.get_layer("global_pooling").output

In [9]:
hidden_layers = [256]

In [10]:
valaence_feat = mobilenet_output
for size in hidden_layers:
    valence_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_valence')(valaence_feat)
outputs_valence = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_valence')(valence_feat) * 4) - 2

arousal_feat = mobilenet_output
for size in hidden_layers:
    arousal_feat = tf.keras.layers.Dense(size, activation = 'relu', name = f'feat_arousal')(arousal_feat)
outputs_arousal = (tf.keras.layers.Dense(1, activation = 'sigmoid', name = 'outputs_arousal')(arousal_feat) * 4) - 2


In [11]:
model = tf.keras.Model(inputs=mobilenet_pretrained.input,
                       outputs=(outputs_valence, outputs_arousal, mobilenet_pretrained.output) , name="mobilenet_train")

In [12]:
model.summary()

Model: "mobilenet_train"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 225, 225, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1 (Conv2D)                 (None, 112, 112, 32  864         ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                    

In [13]:
# Instantiate an optimizer.
optimizer = tf.keras.optimizers.Adam()
# Instantiate a loss function.
MSE_loss = tf.keras.losses.MeanSquaredError()
SCC_loss = tf.keras.losses.SparseCategoricalCrossentropy()

In [14]:
batch_size = 8
epochs = 4

logs = []

In [ ]:
from telegram_notifier import send_message
import time
start_time = time.time()


for epoch in range(epochs):
    total_loss = 0
    total_step = 0
    total_emotion_correct = 0
    total_valence_loss = 0
    total_arousal_loss = 0
    total_emotion_loss = 0
    total_process = 0
    for step, row in enumerate(DataFrameBatchIterator(train_df, batch_size=batch_size)):

        imgs = row.subDirectory_filePath.apply(lambda x: cv2.resize(
            cv2.cvtColor(cv2.imread(x), cv2.COLOR_BGR2RGB), (224, 224)))
        img_array = np.array(list(imgs))

        y_valence = np.array(row.valence)
        y_arousal = np.array(row.arousal)
        y_emotion = np.array(row.expression)

        with tf.GradientTape() as tape:
            logits = model(img_array, training=True)

            pred_valence = logits[0]
            pred_arousal = logits[1]
            pred_emotion = logits[2]

            valence_loss = MSE_loss(y_valence, pred_valence)
            arousal_loss = MSE_loss(y_arousal, pred_arousal)
            emotion_loss = SCC_loss(y_emotion, pred_emotion)
            
            loss = valence_loss + arousal_loss # + emotion_loss

            grads = tape.gradient(loss, model.trainable_weights)

            optimizer.apply_gradients(zip(grads, model.trainable_weights))
        total_loss += float((loss))
        total_step += 1
        
        total_process += len(row)

        valence_loss = float((valence_loss))
        arousal_loss = float((arousal_loss))
        emotion_loss = float((emotion_loss))
        
        total_valence_loss += valence_loss
        total_arousal_loss += arousal_loss
        total_emotion_loss += emotion_loss
        
        emotion_correct = int(sum(pred_emotion.numpy().argmax(axis = 1) == y_emotion))
        total_emotion_correct += emotion_correct
        
        log = {
            'emotion_correct': emotion_correct,
            'valence_loss': valence_loss,
            'arousal_loss': arousal_loss,
            'emotion_loss': emotion_loss,
            'time_take': time.time() - start_time,
            'pred_valence': list(map(lambda x: float(x), np.around(np.array(pred_valence).reshape(-1), 3))),
            'pred_arousal': list(map(lambda x: float(x), np.around(np.array(pred_arousal).reshape(-1), 3))),
        }
        log = json.dumps(log)
        lprint(log)
    save_model_path = f"models/{now_time_string}_{description}_epoch{epoch}_batch_{batch_size}"
    model.save(save_model_path)
    lprint(f"Save {save_model_path}")
    send_message(f"Save {save_model_path}\nepoch {epoch} is finish")
    log = {
        'model_path': save_model_path,
        'total_emotion_correct': total_emotion_correct,
        'total_loss': total_loss,
        'total_valence_loss': total_valence_loss,
        'total_arousal_loss': total_arousal_loss,
        'total_emotion_loss': total_emotion_loss,
        'total_process': total_process,
        'time_take': time.time() - start_time,
    }
    log = json.dumps(log)
    lprint(log)

{"emotion_correct": 1, "valence_loss": 0.8131808042526245, "arousal_loss": 0.7259848117828369, "emotion_loss": NaN, "time_take": 2.113193988800049, "pred_valence": [0.23199999332427979, -0.04399999976158142, 0.12399999797344208, 0.14800000190734863, 0.48500001430511475, 0.30300000309944153, 0.012000000104308128, 0.2800000011920929], "pred_arousal": [-0.029999999329447746, -0.1679999977350235, -0.38499999046325684, 0.012000000104308128, -0.014999999664723873, 0.2070000022649765, -0.8019999861717224, -0.257999986410141]}
{"emotion_correct": 0, "valence_loss": 1.3117731809616089, "arousal_loss": 2.3728036880493164, "emotion_loss": NaN, "time_take": 2.1889045238494873, "pred_valence": [-1.1579999923706055, -1.00600004196167, -1.065000057220459, -0.8370000123977661, -1.2580000162124634, -0.7099999785423279, -1.0329999923706055, -0.6840000152587891], "pred_arousal": [-0.01600000075995922, -0.41200000047683716, -0.12700000405311584, -0.027000000700354576, 0.5040000081062317, -0.18000000715255

{"emotion_correct": 0, "valence_loss": 1.7829698324203491, "arousal_loss": 0.8861191868782043, "emotion_loss": NaN, "time_take": 3.5721330642700195, "pred_valence": [-0.593999981880188, -0.7689999938011169, -0.5249999761581421, -0.49799999594688416, -0.37299999594688416, -0.7950000166893005, -0.628000020980835, -0.5870000123977661], "pred_arousal": [-0.9860000014305115, -1.2489999532699585, -1.2089999914169312, -0.9210000038146973, -0.7039999961853027, -0.9700000286102295, -1.0, -1.0369999408721924]}
{"emotion_correct": 0, "valence_loss": 0.8985456228256226, "arousal_loss": 0.9493727684020996, "emotion_loss": NaN, "time_take": 3.6742520332336426, "pred_valence": [-0.7039999961853027, -0.6800000071525574, -0.4399999976158142, -0.5699999928474426, -0.9049999713897705, -0.28600001335144043, -0.414000004529953, -0.5350000262260437], "pred_arousal": [-0.7960000038146973, -1.1469999551773071, -0.4099999964237213, -0.6389999985694885, -1.4390000104904175, -0.5680000185966492, -0.7960000038146

{"emotion_correct": 0, "valence_loss": 1.5191709995269775, "arousal_loss": 1.174773931503296, "emotion_loss": NaN, "time_take": 5.08226203918457, "pred_valence": [-0.08299999684095383, -0.26600000262260437, -0.1889999955892563, -0.4259999990463257, -0.3619999885559082, -0.1850000023841858, -0.4000000059604645, -0.3529999852180481], "pred_arousal": [-1.0230000019073486, -0.5049999952316284, -0.9309999942779541, -0.5740000009536743, -0.8270000219345093, -0.23100000619888306, -0.703000009059906, -0.625]}
{"emotion_correct": 0, "valence_loss": 0.8016542196273804, "arousal_loss": 0.9264522790908813, "emotion_loss": NaN, "time_take": 5.170802116394043, "pred_valence": [-0.2849999964237213, -0.4350000023841858, -0.3409999907016754, -0.14900000393390656, -0.19499999284744263, -0.3610000014305115, -0.39100000262260437, -0.6480000019073486], "pred_arousal": [-0.5249999761581421, -0.7009999752044678, -0.878000020980835, -0.8479999899864197, -0.8149999976158142, -0.7080000042915344, -0.68699997663

{"emotion_correct": 0, "valence_loss": 1.1864231824874878, "arousal_loss": 1.2242629528045654, "emotion_loss": NaN, "time_take": 6.5795512199401855, "pred_valence": [-0.12999999523162842, -0.3089999854564667, 0.17499999701976776, -0.15299999713897705, -0.26499998569488525, 0.019999999552965164, -0.3400000035762787, 0.1679999977350235], "pred_arousal": [-0.11800000071525574, -0.004000000189989805, 0.02800000086426735, 0.01600000075995922, -0.06499999761581421, 0.15600000321865082, -0.028999999165534973, 0.18400000035762787]}
{"emotion_correct": 0, "valence_loss": 1.3354377746582031, "arousal_loss": 1.0125329494476318, "emotion_loss": NaN, "time_take": 6.665751695632935, "pred_valence": [-0.020999999716877937, -0.10499999672174454, 0.039000000804662704, -0.17800000309944153, -0.20399999618530273, -0.07000000029802322, 0.0020000000949949026, 0.020999999716877937], "pred_arousal": [-0.0010000000474974513, -0.1940000057220459, 0.15199999511241913, -0.06700000166893005, -0.1809999942779541, 

{"emotion_correct": 0, "valence_loss": 0.7358199954032898, "arousal_loss": 0.47178322076797485, "emotion_loss": NaN, "time_take": 8.02692437171936, "pred_valence": [-0.37400001287460327, -0.6930000185966492, -0.5630000233650208, -0.6809999942779541, -0.46799999475479126, -0.652999997138977, -0.6549999713897705, -0.5730000138282776], "pred_arousal": [-0.4779999852180481, -0.8059999942779541, -0.30300000309944153, -0.5870000123977661, -0.49000000953674316, -0.578000009059906, -0.6150000095367432, -0.531000018119812]}
{"emotion_correct": 0, "valence_loss": 0.9552889466285706, "arousal_loss": 1.2779784202575684, "emotion_loss": NaN, "time_take": 8.11076045036316, "pred_valence": [-0.6380000114440918, -0.460999995470047, -0.656000018119812, -0.44999998807907104, -0.8519999980926514, -0.43799999356269836, -0.4869999885559082, -0.5920000076293945], "pred_arousal": [-0.4869999885559082, -0.34599998593330383, -0.6510000228881836, -0.3869999945163727, -0.7080000042915344, -0.45399999618530273, -

{"emotion_correct": 0, "valence_loss": 1.4869097471237183, "arousal_loss": 1.5962200164794922, "emotion_loss": NaN, "time_take": 9.561068296432495, "pred_valence": [-0.3619999885559082, -0.22599999606609344, -0.47600001096725464, -0.33899998664855957, -0.453000009059906, -0.2849999964237213, -0.24199999868869781, -0.27300000190734863], "pred_arousal": [-0.44200000166893005, -0.4959999918937683, -0.6830000281333923, -0.6110000014305115, -0.6840000152587891, -0.5770000219345093, -0.4129999876022339, -0.43299999833106995]}
{"emotion_correct": 0, "valence_loss": 0.6769661903381348, "arousal_loss": 0.34974178671836853, "emotion_loss": NaN, "time_take": 9.653671503067017, "pred_valence": [-0.32199999690055847, -0.3840000033378601, -0.2409999966621399, -0.3959999978542328, -0.21299999952316284, -0.2619999945163727, -0.4000000059604645, -0.3529999852180481], "pred_arousal": [-0.5059999823570251, -0.4779999852180481, -0.5989999771118164, -0.6380000114440918, -0.46399998664855957, -0.61699998378

{"emotion_correct": 0, "valence_loss": 1.208004355430603, "arousal_loss": 0.9793088436126709, "emotion_loss": NaN, "time_take": 11.076772212982178, "pred_valence": [0.1509999930858612, -0.07199999690055847, -0.04399999976158142, -0.2590000033378601, -0.328000009059906, -0.11500000208616257, 0.05299999937415123, -0.1340000033378601], "pred_arousal": [-0.14000000059604645, -0.21199999749660492, -0.2409999966621399, -0.06800000369548798, -0.2750000059604645, -0.08699999749660492, -0.10599999874830246, -0.2529999911785126]}
{"emotion_correct": 1, "valence_loss": 1.2676990032196045, "arousal_loss": 1.1512739658355713, "emotion_loss": NaN, "time_take": 11.147669315338135, "pred_valence": [0.024000000208616257, -0.30399999022483826, -0.22200000286102295, -0.0989999994635582, -0.061000000685453415, -0.32100000977516174, -0.0729999989271164, 0.20200000703334808], "pred_arousal": [-0.050999999046325684, -0.34599998593330383, -0.26499998569488525, -0.15399999916553497, -0.07900000363588333, -0.24

{"emotion_correct": 0, "valence_loss": 1.6017353534698486, "arousal_loss": 1.1177160739898682, "emotion_loss": NaN, "time_take": 12.328140497207642, "pred_valence": [-0.3089999854564667, -0.3580000102519989, -0.2280000001192093, -0.39100000262260437, -0.4169999957084656, -0.5130000114440918, -0.2619999945163727, -0.2290000021457672], "pred_arousal": [-0.30799999833106995, -0.5009999871253967, -0.3160000145435333, -0.7039999961853027, -0.515999972820282, -0.4490000009536743, -0.335999995470047, -0.27900001406669617]}
{"emotion_correct": 0, "valence_loss": 1.1348987817764282, "arousal_loss": 1.07601797580719, "emotion_loss": NaN, "time_take": 12.407004356384277, "pred_valence": [-0.32899999618530273, -0.27000001072883606, -0.5070000290870667, -0.28999999165534973, -0.15700000524520874, -0.30799999833106995, -0.3089999854564667, -0.20399999618530273], "pred_arousal": [-0.43299999833106995, -0.39500001072883606, -0.949999988079071, -0.40799999237060547, -0.3569999933242798, -0.564000010490

{"emotion_correct": 0, "valence_loss": 0.5791620016098022, "arousal_loss": 0.6781256198883057, "emotion_loss": NaN, "time_take": 13.741801977157593, "pred_valence": [-0.24400000274181366, -0.21799999475479126, -0.32899999618530273, -0.30399999022483826, -0.25999999046325684, -0.2800000011920929, -0.22100000083446503, -0.24400000274181366], "pred_arousal": [-0.31299999356269836, -0.2669999897480011, -0.2759999930858612, -0.3529999852180481, -0.460999995470047, -0.3190000057220459, -0.34599998593330383, -0.2770000100135803]}
{"emotion_correct": 0, "valence_loss": 1.488377332687378, "arousal_loss": 1.3528826236724854, "emotion_loss": NaN, "time_take": 13.818182229995728, "pred_valence": [-0.16899999976158142, -0.3160000145435333, -0.19200000166893005, -0.2879999876022339, -0.0989999994635582, -0.3499999940395355, -0.03999999910593033, -0.27000001072883606], "pred_arousal": [-0.1550000011920929, -0.33799999952316284, -0.2160000056028366, -0.1940000057220459, -0.2029999941587448, -0.3199999

{"emotion_correct": 0, "valence_loss": 1.1876866817474365, "arousal_loss": 0.5194448232650757, "emotion_loss": NaN, "time_take": 15.274280548095703, "pred_valence": [-0.33000001311302185, -0.46000000834465027, -0.3499999940395355, -0.3149999976158142, -0.3499999940395355, -0.39500001072883606, -0.328000009059906, -0.30300000309944153], "pred_arousal": [-0.38199999928474426, -0.6740000247955322, -0.48899999260902405, -0.3779999911785126, -0.382999986410141, -0.640999972820282, -0.46299999952316284, -0.31700000166893005]}
{"emotion_correct": 0, "valence_loss": 1.2429431676864624, "arousal_loss": 1.2614719867706299, "emotion_loss": NaN, "time_take": 15.381113767623901, "pred_valence": [-0.5090000033378601, -0.335999995470047, -0.30300000309944153, -0.43799999356269836, -0.49300000071525574, -0.2800000011920929, -0.3050000071525574, -0.35499998927116394], "pred_arousal": [-0.6669999957084656, -0.44999998807907104, -0.3700000047683716, -0.5419999957084656, -0.5950000286102295, -0.4140000045

In [ ]:
json.dumps(log)